In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Have we got all the forcing we need?

Run plan from email:

    *notes from metoffice (TJ notes in italics):*

1. 1950 to 2100 using both SSP126 and SSP370 with standard UKESM1.0 : These runs are done, yes just need to provide the data.
    - *call SSP126 **1a** and SSP370 **1b***
2. 1950 to 2100 using both SSP126 and SSP370 each with ODS fixed at 1950 values
    - *call SSP126 **2a** and SSP370 **2b***
3. 1985 to 2100 using both SSP126 and SSP370 each with ODS fixed at 1985 values
    - *call SSP126 **3a** and SSP370 **3b***

    
File structure of things received from the met office:

       |-ozone
       |---u-be682_orgGRD TJ>1a/1b hist
       |---u-bc370_orgGRD TJ>1a/1b hist
       |-fixed_ozone
       |---ssp126
       |-----u-cj484_ssp126_1990start1990ozone TJ>3a future
       |---hist
       |-----u-bc370_hist  TJ>1a/1b hist
       |-----u-cj198_hist_1950start1950ozone TJ>2a/2b hist
       |-----u-cj200_hist_1990start1990ozone TJ>3a/3b hist
       |---ssp370
       |-----u-ce417_ssp370 TJ>1b future
       |-----u-cj504_ssp370_1990start1990ozone TJ>2b future
       
       


In [3]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload

import matplotlib.path as mpath
import glob


## 1 historical (ozone left to evolve naturally)

In [4]:
tdir = '/gpfs/data/greenocean/software/resources/'

tdir2 = 'MetOffice/fixed_ozone/hist/u-bc370_hist/'
verbose = True

strs = ['bulk','dust','rad','river','slp','wind']

print(f'checking files for dir {tdir}{tdir2}')
yrs = []
yrs_allgone = []
for y in range(1950,2015):
    tyr = y

    yrstr = f'{tdir}{tdir2}*_y{y}*.nc'
    twfy = glob.glob(yrstr)
    nofilfy = len(twfy)
    
    if (nofilfy == 0):
        print(f'ALLGONEFORTHISYEAR!! {tyr}')
        yrs_allgone.append(tyr)
        yrs.append(tyr)    
    
    else:
        for s in range(0,len(strs)):

            tch = f'{tdir}{tdir2}*{strs[s]}_y{y}*.nc'

            tw = glob.glob(tch)
            nofil = len(tw)
            if (strs[s]=='slp') & (nofil != 4):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
                if tyr not in yrs:
                    yrs.append(tyr)
            elif ((strs[s]!='slp') &(nofil != 12)):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
        #cj198_wind_y1980
                if tyr not in yrs:
                    yrs.append(tyr)
        
print(yrs)
print(yrs_allgone)
# grep -l "string" * | wc -l    

checking files for dir /gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/hist/u-bc370_hist/
[]
[]


####  future scenarios: 1a, natl ozone and ssp 1-2.6, 1b natl ozone and ssp 3-7.0
# 1a is missing entirely (scen ssp 1-2.6)
# 1b - future, scenario SSP 3-7.0

In [5]:
tdir = '/gpfs/data/greenocean/software/resources/'

tdir2 = 'MetOffice/fixed_ozone/ssp370/u-ce417_ssp370/'
verbose = True

strs = ['bulk','dust','rad','river','slp','wind']

print(f'checking files for dir {tdir}{tdir2}')
yrs = []
yrs_allgone = []
for y in range(2015,2101):
    tyr = y

    yrstr = f'{tdir}{tdir2}*_y{y}*.nc'
    twfy = glob.glob(yrstr)
    nofilfy = len(twfy)
    
    if (nofilfy == 0):
        print(f'ALLGONEFORTHISYEAR!! {tyr}')
        yrs_allgone.append(tyr)
        yrs.append(tyr)    
    
    else:
        for s in range(0,len(strs)):

            tch = f'{tdir}{tdir2}*{strs[s]}_y{y}*.nc'

            tw = glob.glob(tch)
            nofil = len(tw)
            if (strs[s]=='slp') & (nofil != 4):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
                if tyr not in yrs:
                    yrs.append(tyr)
            elif ((strs[s]!='slp') &(nofil != 12)):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
        #cj198_wind_y1980
                if tyr not in yrs:
                    yrs.append(tyr)
        
print(yrs)
print(yrs_allgone)
# grep -l "string" * | wc -l    

checking files for dir /gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/ssp370/u-ce417_ssp370/
MISSING FILES! year: 2056, vartype: bulk
MISSING FILES! year: 2056, vartype: dust
MISSING FILES! year: 2056, vartype: rad
MISSING FILES! year: 2056, vartype: river
MISSING FILES! year: 2056, vartype: wind
MISSING FILES! year: 2057, vartype: bulk
MISSING FILES! year: 2057, vartype: dust
MISSING FILES! year: 2057, vartype: rad
MISSING FILES! year: 2057, vartype: river
MISSING FILES! year: 2057, vartype: wind
MISSING FILES! year: 2058, vartype: bulk
MISSING FILES! year: 2058, vartype: dust
MISSING FILES! year: 2058, vartype: rad
MISSING FILES! year: 2058, vartype: river
MISSING FILES! year: 2058, vartype: wind
MISSING FILES! year: 2059, vartype: bulk
MISSING FILES! year: 2059, vartype: dust
MISSING FILES! year: 2059, vartype: rad
MISSING FILES! year: 2059, vartype: river
MISSING FILES! year: 2059, vartype: wind
MISSING FILES! year: 2060, vartype: bulk
MISSING FILES! year: 2060, var

# 2- historical (ozone left at 1950 values)

In [6]:
tdir = '/gpfs/data/greenocean/software/resources/'

tdir2 = 'MetOffice/fixed_ozone/hist/u-cj198_hist_1950start1950ozone/'
verbose = True

strs = ['bulk','dust','rad','river','slp','wind']

print(f'checking files for dir {tdir}{tdir2}')
yrs = []
yrs_allgone = []
for y in range(1950,2015):
    tyr = y

    yrstr = f'{tdir}{tdir2}*_y{y}*.nc'
    twfy = glob.glob(yrstr)
    nofilfy = len(twfy)
    
    if (nofilfy == 0):
        print(f'ALLGONEFORTHISYEAR!! {tyr}')
        yrs_allgone.append(tyr)
        yrs.append(tyr)    
    
    else:
        for s in range(0,len(strs)):

            tch = f'{tdir}{tdir2}*{strs[s]}_y{y}*.nc'

            tw = glob.glob(tch)
            nofil = len(tw)
            if (strs[s]=='slp') & (nofil != 4):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
                if tyr not in yrs:
                    yrs.append(tyr)
            elif ((strs[s]!='slp') &(nofil != 12)):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
        #cj198_wind_y1980
                if tyr not in yrs:
                    yrs.append(tyr)
      
print('somegone')
print(yrs)
print('allgone')
print(yrs_allgone)
# grep -l "string" * | wc -l    

checking files for dir /gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/hist/u-cj198_hist_1950start1950ozone/
MISSING FILES! year: 2014, vartype: dust
somegone
[2014]
allgone
[]


#### future scenario 2 ODS fixed at 1950 values
# 2a (scen 1-2.6) missing entirely
# 2b (scen 3-7.0) missing entirely


In [11]:
tdir = '/gpfs/data/greenocean/software/resources/'


tdir2 = 'MetOffice/fixed_ozone/ssp370/u-cj881_ssp370_1950start1950ozone//'
verbose = True

strs = ['bulk','dust','rad','river','slp','wind']

print(f'checking files for dir {tdir}{tdir2}')
yrs = []
yrs_allgone = []
for y in range(2015,2101):
    tyr = y

    yrstr = f'{tdir}{tdir2}*_y{y}*.nc'
    twfy = glob.glob(yrstr)
    nofilfy = len(twfy)
    
    if (nofilfy == 0):
        print(f'ALLGONEFORTHISYEAR!! {tyr}')
        yrs_allgone.append(tyr)
        yrs.append(tyr)    
    
    else:
        for s in range(0,len(strs)):

            tch = f'{tdir}{tdir2}*{strs[s]}_y{y}*.nc'

            tw = glob.glob(tch)
            nofil = len(tw)
            if (strs[s]=='slp') & (nofil != 4):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
                if tyr not in yrs:
                    yrs.append(tyr)
            elif ((strs[s]!='slp') &(nofil != 12)):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
        #cj198_wind_y1980
                if tyr not in yrs:
                    yrs.append(tyr)
      
print('somegone')
print(yrs)
print('allgone')
print(yrs_allgone)

checking files for dir /gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/ssp370/u-cj881_ssp370_1950start1950ozone//
MISSING FILES! year: 2094, vartype: bulk
MISSING FILES! year: 2094, vartype: dust
MISSING FILES! year: 2094, vartype: rad
MISSING FILES! year: 2094, vartype: wind
MISSING FILES! year: 2095, vartype: bulk
MISSING FILES! year: 2095, vartype: dust
MISSING FILES! year: 2095, vartype: rad
MISSING FILES! year: 2095, vartype: wind
MISSING FILES! year: 2096, vartype: bulk
MISSING FILES! year: 2096, vartype: dust
MISSING FILES! year: 2096, vartype: rad
MISSING FILES! year: 2096, vartype: wind
MISSING FILES! year: 2097, vartype: bulk
MISSING FILES! year: 2097, vartype: dust
MISSING FILES! year: 2097, vartype: rad
MISSING FILES! year: 2097, vartype: wind
MISSING FILES! year: 2098, vartype: bulk
MISSING FILES! year: 2098, vartype: dust
MISSING FILES! year: 2098, vartype: rad
MISSING FILES! year: 2098, vartype: wind
MISSING FILES! year: 2099, vartype: bulk
MISSING FILES! 

# 3 historical - ODS evolves naturally until 1990, then fixed at 1990 values (so only need files 1990-2014)

In [12]:
tdir = '/gpfs/data/greenocean/software/resources/'

tdir2 = 'MetOffice/fixed_ozone/hist/u-cj200_hist_1990start1990ozone/'
verbose = True

strs = ['bulk','dust','rad','river','slp','wind']

print(f'checking files for dir {tdir}{tdir2}')
yrs = []
yrs_allgone = []
for y in range(1990,2015):
    tyr = y

    yrstr = f'{tdir}{tdir2}*_y{y}*.nc'
    twfy = glob.glob(yrstr)
    nofilfy = len(twfy)
    
    if (nofilfy == 0):
        print(f'ALLGONEFORTHISYEAR!! {tyr}')
        yrs_allgone.append(tyr)
        yrs.append(tyr)    
    
    else:
        for s in range(0,len(strs)):

            tch = f'{tdir}{tdir2}*{strs[s]}_y{y}*.nc'

            tw = glob.glob(tch)
            nofil = len(tw)
            if (strs[s]=='slp') & (nofil != 4):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
                if tyr not in yrs:
                    yrs.append(tyr)
            elif ((strs[s]!='slp') &(nofil != 12)):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
        #cj198_wind_y1980
                if tyr not in yrs:
                    yrs.append(tyr)
      
print('somegone')
print(yrs)
print('allgone')
print(yrs_allgone)
# grep -l "string" * | wc -l    

checking files for dir /gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/hist/u-cj200_hist_1990start1990ozone/
somegone
[]
allgone
[]


## 3 future, ods fixed at 1990 values
## 3a SSP 1-2.6


In [13]:
tdir = '/gpfs/data/greenocean/software/resources/'


tdir2 = 'MetOffice/fixed_ozone/ssp126/u-cj484_ssp126_1990start1990ozone/'
verbose = True

strs = ['bulk','dust','rad','river','slp','wind']

print(f'checking files for dir {tdir}{tdir2}')
yrs = []
yrs_allgone = []
for y in range(2015,2101):
    tyr = y

    yrstr = f'{tdir}{tdir2}*_y{y}*.nc'
    twfy = glob.glob(yrstr)
    nofilfy = len(twfy)
    
    if (nofilfy == 0):
        print(f'ALLGONEFORTHISYEAR!! {tyr}')
        yrs_allgone.append(tyr)
        yrs.append(tyr)    
    
    else:
        for s in range(0,len(strs)):

            tch = f'{tdir}{tdir2}*{strs[s]}_y{y}*.nc'

            tw = glob.glob(tch)
            nofil = len(tw)
            if (strs[s]=='slp') & (nofil != 4):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
                if tyr not in yrs:
                    yrs.append(tyr)
            elif ((strs[s]!='slp') &(nofil != 12)):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
        #cj198_wind_y1980
                if tyr not in yrs:
                    yrs.append(tyr)
      
print('somegone')
print(yrs)
print('allgone')
print(yrs_allgone)

checking files for dir /gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/ssp126/u-cj484_ssp126_1990start1990ozone/
somegone
[]
allgone
[]


## 3b SSP 3-7.0


In [14]:
tdir = '/gpfs/data/greenocean/software/resources/'


tdir2 = 'MetOffice/fixed_ozone/ssp370/u-cj504_ssp370_1990start1990ozone/'
verbose = True

strs = ['bulk','dust','rad','river','slp','wind']

print(f'checking files for dir {tdir}{tdir2}')
yrs = []
yrs_allgone = []
for y in range(2015,2101):
    tyr = y

    yrstr = f'{tdir}{tdir2}*_y{y}*.nc'
    twfy = glob.glob(yrstr)
    nofilfy = len(twfy)
    
    if (nofilfy == 0):
        print(f'ALLGONEFORTHISYEAR!! {tyr}')
        yrs_allgone.append(tyr)
        yrs.append(tyr)    
    
    else:
        for s in range(0,len(strs)):

            tch = f'{tdir}{tdir2}*{strs[s]}_y{y}*.nc'

            tw = glob.glob(tch)
            nofil = len(tw)
            if (strs[s]=='slp') & (nofil != 4):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
                if tyr not in yrs:
                    yrs.append(tyr)
            elif ((strs[s]!='slp') &(nofil != 12)):
                print(f'MISSING FILES! year: {tyr}, vartype: {strs[s]}')
                # print('files available for this year and vartype:')
                # print(tw)
        #cj198_wind_y1980
                if tyr not in yrs:
                    yrs.append(tyr)
      
print('somegone')
print(yrs)
print('allgone')
print(yrs_allgone)

checking files for dir /gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/ssp370/u-cj504_ssp370_1990start1990ozone/
MISSING FILES! year: 2039, vartype: river
MISSING FILES! year: 2040, vartype: river
MISSING FILES! year: 2041, vartype: river
MISSING FILES! year: 2042, vartype: river
MISSING FILES! year: 2043, vartype: river
MISSING FILES! year: 2044, vartype: river
MISSING FILES! year: 2045, vartype: river
MISSING FILES! year: 2046, vartype: bulk
MISSING FILES! year: 2046, vartype: dust
MISSING FILES! year: 2046, vartype: rad
MISSING FILES! year: 2046, vartype: river
MISSING FILES! year: 2046, vartype: wind
MISSING FILES! year: 2047, vartype: bulk
MISSING FILES! year: 2047, vartype: dust
MISSING FILES! year: 2047, vartype: rad
MISSING FILES! year: 2047, vartype: river
MISSING FILES! year: 2047, vartype: wind
MISSING FILES! year: 2048, vartype: bulk
MISSING FILES! year: 2048, vartype: dust
MISSING FILES! year: 2048, vartype: rad
MISSING FILES! year: 2048, vartype: river
MISS